In [41]:
import ipytest
# ipytest.autoconfig(addopts=["-x", "--pdb"])
ipytest.autoconfig()
%run 06_new_node_tree_cursor_implementation.ipynb

All classes are loaded


### Tests for Node Class

In [42]:
%%ipytest
import pytest

def test_node_instantiation():
    assert Node(name="a") is not None
    assert Node(name="a", visited=True) is not None
    assert Node(name=1) is not None
    assert Node(name=21) is not None

def test_node_name_gt_one_char():
    with pytest.raises(ValueError):
        nameerror = Node("ab")

def test_node_eq():
    n1 = Node(name="a")
    n2 = Node(name="a")
    
    assert n1 == n2
    n2.visited = True 
    assert n1 == n2 # if dataclass __eq__ is not overidden, this will fail

def test_node_add_neighbors():
    n1 = Node(name="a")
    n2 = Node(name="b")
    n1.add_neighbor(n2)
    
    assert n2 in n1.neighbors

def test_node_visit():
    n1 = Node(name="a")
    n1.visit()

def test_node_unvisit():
    n1 = Node(name="a")
    n1.visited = True
    n1.unvisit()
    
    assert n1.visited == False

......                                                                                       [100%]
6 passed in 0.01s


### Tests for Tree Class

In [43]:
%%ipytest
import pytest

@pytest.fixture
def tree():
    n1 = Node(name="a")
    n2 = Node(name="b")
    tree = Tree(name="test_tree")
    tree.nodes.append(n1)
    tree.nodes.append(n2)
    yield tree

def test_tree_instantiation():
    assert Tree(name="test_tree") is not None

def test_tree_add_node_str(tree):
    n3 = Node(name="c")
    tree.add_node(n3)
    assert n3 in tree.nodes
    n4 = Node(name="a")
    # If we try to add a note with the same name of another
    # node, a ValueError should be raised
    with pytest.raises(ValueError):
        tree.add_node(n3)

def test_tree_add_node_ints(tree):
    n3 = Node(name=1)
    tree.add_node(n3)
    assert n3 in tree.nodes
    n4 = Node(name=1)
    # If we try to add a note with the same name of another
    # node, a ValueError should be raised
    with pytest.raises(ValueError):
        tree.add_node(n3)

def test_tree_has_node_strs(tree):
    assert tree.has_node('a')

def test_tree_has_node_ints(tree):
    n5 = Node(1)
    n6 = Node(2)
    tree.nodes.append(n5)
    tree.nodes.append(n6)
    assert tree.has_node(1)
    assert tree.has_node(2)

def test_tree_get_node(tree):
    assert tree.get_node('a') == Node(name="a")
    assert tree.get_node('z') == None
    n5 = Node(1)
    n6 = Node(2)
    tree.nodes.append(n5)
    tree.nodes.append(n6)
    assert tree.get_node(1) == Node(name=1)
    assert tree.get_node(2) == Node(name=2)
    

def test_tree_from_dict_str():
    data = {
        "a": ["b", "c"],
        "b": ["d"],
        "c": [],
        "d": [],
    }
    
    tree = Tree.from_dict("test_tree", data)
    assert tree is not None
    assert Node("a") in tree.nodes
    assert Node("b") in tree.nodes
    assert Node("c") in tree.nodes
    assert Node("d") in tree.nodes
    neighbor = tree.nodes[1] # b node
    assert neighbor in tree.nodes[0].neighbors # a node

def test_tree_from_dict_int():
    data = {
        1: [2, 3],
        2: [4],
        3: [],
        4: [],
    }
    
    tree = Tree.from_dict("test_tree", data)
    assert tree is not None
    assert Node(1) in tree.nodes
    assert Node(2) in tree.nodes
    assert Node(3) in tree.nodes
    assert Node(4) in tree.nodes
    neighbor = tree.nodes[1] # 2 node
    assert neighbor in tree.nodes[0].neighbors # 1 node
    
def test_tree_from_edges():
    edges = [
        ['a', 'b'],
        ['b', 'c'],
        ['c', 'a']
    ]
    
    tree = Tree.from_edges("test_tree", edges)
    assert tree is not None
    assert Node("a") in tree.nodes
    assert Node("b") in tree.nodes
    assert Node("c") in tree.nodes
    neighbor1 = tree.nodes[0] # a
    neighbor2 = tree.nodes[1] # b
    neighbor3 = tree.nodes[2] # c
    assert neighbor1 in neighbor2.neighbors # a in b.neighbors
    assert neighbor2 in neighbor1.neighbors # b in a.neighbors
    assert neighbor3 in neighbor1.neighbors # c in a.neighbors

def test_tree_reset_nodes(tree):
    tree.nodes[0].visited = True
    tree.nodes[1].visited = True
    tree.reset_nodes()
    assert tree.nodes[0].visited == False
    assert tree.nodes[1].visited == False

..........                                                                                   [100%]
10 passed in 0.02s


### Tests for Stack Class

In [44]:
%%ipytest

def test_stack_instantiation():
    assert Stack() is not None
    assert Stack().nodes == []

def test_stack_push():
    stack = Stack()
    n1 = Node(name="a")
    n2 = Node(name="b")
    n3 = Node(name="c")
    stack.push(n1)
    assert stack.nodes[0] == n1
    stack.push(n2)
    assert stack.nodes[1] == n2

def test_stack_pop():
    # A stack should pop the last inserted element (LIFO)
    stack = Stack()
    n1 = Node(name="a")
    n2 = Node(name="b")
    n3 = Node(name="c")
    stack.push(n1) # First inserted
    stack.push(n2)
    stack.push(n3) # Last inserted
    assert stack.pop() == n3 # Last inserted
    assert stack.pop() == n2
    assert stack.pop() == n1 # First inserted

...                                                                                          [100%]
3 passed in 0.01s


### Tests for Queue Class

In [45]:
%%ipytest

def test_queue_instantiation():
    assert Queue() is not None
    assert Queue().nodes == []
    
def test_queue_push():
    queue = Queue()
    n1 = Node(name="a")
    n2 = Node(name="b")
    n3 = Node(name="c")
    queue.push(n1)
    assert queue.nodes[0] == n1
    queue.push(n2)
    assert queue.nodes[1] == n2
    
def test_queue_pop():
    # A stack should pop the last inserted element (LIFO)
    queue = Queue()
    n1 = Node(name="a")
    n2 = Node(name="b")
    n3 = Node(name="c")
    queue.push(n1) # First inserted
    queue.push(n2)
    queue.push(n3) # Last inserted
    assert queue.pop() == n1 # First inserted
    assert queue.pop() == n2
    assert queue.pop() == n3 # Last inserted

...                                                                                          [100%]
3 passed in 0.01s


### Tests for Cursor Class

In [48]:
%%ipytest

import pytest

@pytest.fixture
def tree_for_walk():
    data = {
        "a": ["b", "c"],
        "b": ["d", "e"],
        "c": ["f"],
        "d": ["g", "h"],
        "e": ["g", "i"],
        "f": ["i"],
        "g": [],
        "h": [],
        "i": []
    }    
    yield Tree.from_dict("test_tree", data)

@pytest.fixture
def cyclical_tree():
    edges = [
        ['i', 'j'],
        ['k', 'i'],
        ['k', 'j'],
        ['m', 'k'],
        ['k', 'l'],
        ['o', 'n']
    ]

    yield Tree.from_edges("test_tree", edges) 

@pytest.fixture
def components_tree():
    graph = {
        0: [8, 1, 5],
        1: [0],
        5: [0, 8],
        8: [0, 5],
        2: [3, 4],
        3: [2, 4],
        4: [3, 2]
    }

@pytest.fixture
def stack():
    yield Stack()

@pytest.fixture
def queue():
    yield Queue()
    
def test_cursor_instantiation(tree_for_walk, stack):
    cursor = Cursor(tree_for_walk, stack)
    assert cursor is not None
    assert cursor.tree is not None
    assert cursor.controller is not None
    
def test_cursor_walk_stack(tree_for_walk, stack):
    cursor = Cursor(tree_for_walk, stack)
    assert cursor.walk("a") == ['a', 'c', 'f', 'i', 'b', 'e', 'g', 'd', 'h']
    
def test_cursor_walk_queue(tree_for_walk, queue):
    cursor = Cursor(tree_for_walk, queue)
    assert cursor.walk("a") == ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']

def test_cursor_has_path_stack(cyclical_tree, stack):
    cursor = Cursor(cyclical_tree, stack)
    assert cursor.has_path("i", "n") is False
    assert cursor.has_path("i", "k") is True

def test_cursor_has_path_queue(cyclical_tree, queue):
    cursor = Cursor(cyclical_tree, queue)
    assert cursor.has_path("i", "n") is False
    assert cursor.has_path("i", "k") is True

def test_cursor_count_islands(components_tree, stack):
    cursor = Cursor(components_tree, stack)
    assert cursor.components() == 2

.....F                                                                                       [100%]
============================================= FAILURES =============================================
______________________________________ test_cursor_components ______________________________________

components_tree = None, stack = Stack(nodes=[])

    def test_cursor_components(components_tree, stack):
        cursor = Cursor(components_tree, stack)
>       assert cursor.components() == 2

/tmp/ipykernel_14397/2911580284.py:77: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

self = Cursor(tree=None, controller=Stack(nodes=[]), current=None)

    def components(self) -> int:
        # import ipdb; ipdb.set_trace()
    
>       for node in self.tree.nodes:
E       AttributeError: 'NoneType' object has no attribute 'nodes'

/tmp/ipykernel_14397/753379634.py:61: AttributeError
===================================== short test summary 